### Figure 3C - Early vs Late Capsule Scatter Plot

Here we look at a read by read scatter plot of the early capsule sample vs the late capsule sample to see if the encapsulated reads have changed significantly while the background cellular context is changing. 

In [33]:
import pandas as pd
import numpy as np
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg
output_notebook()
from bokeh.plotting import figure
from bokeh.layouts import column
import datetime

Loading BokehJS ...

First we will read our data frames into a file:

In [34]:
df_dir = '../../../Processed_Data_Files/dataframe_files/230712LiA/snapshot/'
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt'):
        name = file[:]
        name = name[:name.find('_D')]
        new_df  = pd.read_csv(''.join([df_dir,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = new_df
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')

# a name dictionary for converting between the datafram name and a form nicer for plots:
names_dict = {'Before-Ind': 'Before Induction', 'Early-Cap': "Early Capsule",
              'Early-Lys': "Early Lysate", 'Late-Cap': "Late Capsule",
              'Late-Lys': "Late Lysate", 'Mid-Stress': "Mid-Stress, Total Lysate",
              'No-Ind': "No Induction", 'Post-Ind': "Post Induction, Total Lysate"}


Here is the list of unique samples for which there are dataframe files: 

['Early_Cap', 'Early_Lys', 'Late_Cap', 'Late_Lys_2', 'Late_Lys']



Some helper functions to help us with the plotting:

In [35]:
def into_RPM_theshold_remove_cap(df_dict, name):
    '''This felper function gets the dataframe for a particular experiment and converts its reads into RPM'''
    sample_df = df_dict[name].copy()
    sample_df.drop(sample_df[sample_df['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
    sample_df.drop(sample_df[sample_df['Name'] == 'Capsule'].index, inplace = True) # currently dropping capsule reads from plot
    sample_df.drop(sample_df[sample_df['Name'] == 'Capsule_rev'].index, inplace = True) # currently dropping capsule reads from plot
    sample_df.drop(sample_df[sample_df['Name'] == 'lacI'].index, inplace = True) # currently dropping lacI from plot
    sample_df.drop(sample_df[sample_df['Counts'] <= 50].index, inplace = True)
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df

Now we will plot a simple scatter plot comparing the early and late Lysate samples on a read by read basis:

In [56]:
x = 'Early_Cap' # Early encapsulated reads
y = 'Late_Cap' # Late encapsulated reads

x_df = into_RPM_theshold_remove_cap(df_dict, x)
y_df = into_RPM_theshold_remove_cap(df_dict, y)
plot_df = x_df.merge(y_df)
plot_df.drop(plot_df[plot_df['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
plot_df.drop(plot_df[plot_df[x] == 0].index, inplace = True)
plot_df.drop(plot_df[plot_df[y] == 0].index, inplace = True)
plot_df['Log_Dif'] = np.log10(plot_df[x]) - np.log10(plot_df[y])

p = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 385, height = 355,
    output_backend = "svg", tooltips = [("Gene", "@Name")]
)


p.axis.ticker = [10**0, 10**2, 10**4, 10**6]
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_size = "20px"
p.xaxis.visible = False

p.background_fill_color = "whitesmoke"
p.grid.grid_line_color = "white"
p.grid.grid_line_width = 4

within_order_mag = plot_df[np.abs(plot_df['Log_Dif']) < 1]
greater_than_order_mag = plot_df[np.abs(plot_df['Log_Dif']) >= 1]

p.circle(x=x, y=y, source=within_order_mag, size=7, fill_alpha=0.8, line_alpha=0, color="orange", legend_label = "MTC")
p.circle(x=x, y=y, source=greater_than_order_mag, size=7, fill_alpha=0.8, line_alpha=0, color="orangered",)
p.line(x=[10**(1.), 10**5], y=[10**(1.), 10**5], color = "black", width = 3.5)
p.line(x=[10*10**(1.), 10**5], y=[10**(1.), 10**4], color = "black", width = 3.5)
p.line(x=[10**(1.), 10**4], y=[10**(2.), 10**5], color = "black", width = 3.5)

p.legend.label_text_font_size = '20pt'
p.legend.label_text_font_style = "normal"
p.legend.location = (5, 290)
p.legend.orientation = "horizontal"
p.toolbar_location = None

show(p)
export_svg(p, filename = f'./2_MTC_{datetime.date.today()}.svg')

['./2_MTC_2023-08-14.svg']

In [66]:
p_insert = figure(width = 150, height = 100,
          output_backend = "svg")

# Histogram
bins = np.linspace(-1, 1, 10)
plt_hist, plt_edges = np.histogram(plot_df['Log_Dif'], density=False, bins=bins)
p_insert.quad(top = plt_hist, bottom = 0, left = plt_edges[:-1], right = plt_edges[1:],
         fill_color="steelblue", line_color="white", fill_alpha = 1,
         )
p_insert.axis.axis_label_text_font_style = "normal"
p_insert.axis.major_label_text_font = "arial"
p_insert.axis.major_label_text_font_size = "15px"
p_insert.yaxis.visible = False
p_insert.yaxis.ticker = [1000, 2000]
p_insert.xaxis.ticker = [-1, -.5, 0, .5, 1]
p_insert.xaxis.axis_label ="log-10 ratio"
p_insert.axis.axis_label_text_font = "arial"
p_insert.axis.axis_label_text_font_size = "15px"

p_insert.background_fill_color = None
p_insert.grid.grid_line_color = None
p_insert.grid.grid_line_width = 4
p_insert.outline_line_color = None

show(p_insert)
export_svg(p_insert, filename = f'2_MTC_Desntiry_Insert_{datetime.date.today()}.svg')

['2_MTC_Desntiry_Insert_2023-08-14.svg']

In [57]:
x2 = 'Early_Lys' # Early whole cell lysate reads
y2 = 'Late_Lys' # Late whole cell lysate reads

x2_df = into_RPM_theshold_remove_cap(df_dict, x2)
y2_df = into_RPM_theshold_remove_cap(df_dict, y2)
plot_df2 = x2_df.merge(y2_df)
plot_df2.drop(plot_df2[plot_df2['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
plot_df2.drop(plot_df2[plot_df2[x2] == 0].index, inplace = True)
plot_df2.drop(plot_df2[plot_df2[y2] == 0].index, inplace = True)
plot_df2['Log_Dif'] = np.log10(plot_df2[x2]) - np.log10(plot_df2[y2])

p2 = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 385, height = 355,
    output_backend = "svg", tooltips = [("Gene", "@Name")]
)

p2.axis.ticker = [10**0, 10**2, 10**4, 10**6]
p2.axis.major_label_text_font = "arial"
p2.axis.major_label_text_font_size = "20px"
p2.xaxis.visible = False

p2.background_fill_color = "whitesmoke"
p2.grid.grid_line_color = "white"
p2.grid.grid_line_width = 4

within_order_mag2 = plot_df2[np.abs(plot_df2['Log_Dif']) < 1]
greater_than_order_mag2 = plot_df2[np.abs(plot_df2['Log_Dif']) >= 1]

p2.circle(x=x2, y=y2, source=within_order_mag2, size=7, fill_alpha=0.8, line_alpha=0, color="darkcyan", legend_label = "Lysate")
p2.circle(x=x2, y=y2, source=greater_than_order_mag2, size=7, fill_alpha=0.8, line_alpha=1, color="darkcyan", line_color = "orangered", line_width = 2)
p2.line(x=[10**(1.), 10**5], y=[10**(1.), 10**5], color = "black", width = 3.5)
p2.line(x=[10*10**(1.), 10**5], y=[10**(1.), 10**4], color = "black", width = 3.5)
p2.line(x=[10**(1.), 10**4], y=[10**(2.), 10**5], color = "black", width = 3.5)

p2.legend.label_text_font_size = '20pt'
p2.legend.label_text_font_style = "normal"
p2.legend.location = (5, 290)
p2.legend.orientation = "horizontal"
p2.toolbar_location = None

show(p2)
export_svg(p2, filename = f'./2_Lys_{datetime.date.today()}.svg')

['./2_Lys_2023-08-14.svg']

In [64]:
p2_insert = figure(width = 150, height = 100,
          output_backend = "svg")

# Histogram
bins = np.linspace(-1, 1, 10)
plt_hist, plt_edges = np.histogram(plot_df2['Log_Dif'], density=False, bins=bins)
p2_insert.quad(top = plt_hist, bottom = 0, left = plt_edges[:-1], right = plt_edges[1:],
         fill_color="steelblue", line_color="white", fill_alpha = 1,
         )
p2_insert.axis.axis_label_text_font_style = "normal"
p2_insert.axis.major_label_text_font = "arial"
p2_insert.axis.major_label_text_font_size = "15px"
p2_insert.yaxis.visible = False
p2_insert.yaxis.ticker = [1000, 2000]
p2_insert.xaxis.ticker = [-1, -.5, 0, .5, 1]
p2_insert.xaxis.axis_label ="log-10 ratio"
p2_insert.axis.axis_label_text_font = "arial"
p2_insert.axis.axis_label_text_font_size = "15px"

p2_insert.background_fill_color = None
p2_insert.grid.grid_line_color = None
p2_insert.grid.grid_line_width = 4
p2_insert.outline_line_color = None

show(p2_insert)
export_svg(p2_insert, filename = f'2_Lys_Desntiry_Insert_{datetime.date.today()}.svg')

['2_Lys_Desntiry_Insert_2023-08-14.svg']

In [58]:
p3 = column(p, p2)
show(p3)
export_svg(p3, filename = f'./2_Stacked_{datetime.date.today()}.svg')

['./2_Stacked_2023-08-14.svg']

In [43]:
print(max(np.abs(plot_df['Log_Dif'])))
print(10**(max(np.abs(plot_df['Log_Dif']))))
print(len(greater_than_order_mag))

0.6999434247655174
5.011219485738238
0


In [31]:
print(float(plot_df.loc[plot_df['Name'] == "tnaA"][y]/plot_df.loc[plot_df['Name'] == "tnaA"][x]))

1.4747650197456919


In [32]:
print(np.corrcoef(np.log10(plot_df[x]), np.log10(plot_df[y]))[0,1])

0.9924583659353944
